In [71]:
import matplotlib.pyplot as plt
from torch.utils import data
import numpy as np
from tqdm import tqdm
import random
from visualisationMetrics import *
from dataLoader import *
from utils.utils import to_pkl, from_pkl, js_divergence, fid
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import defaultdict

In [73]:
NSETS = 20
LOGIR_NAME = 'var'

if not os.path.exists(f'results/{LOGIR_NAME}/'):
    os.mkdir(f'results/{LOGIR_NAME}/')

# Load results

### Load real data

In [48]:
real_data = load_dataset(data_mode='Train')
real_dataloader = data.DataLoader(real_data, batch_size=1, num_workers=1, shuffle=True)

real_paths = []
real_conds = []

for i, (cond, sim) in enumerate(real_dataloader):
    sim = sim.cpu().detach().numpy()
    sim = sim.reshape(sim.shape[1], sim.shape[3])
    real_paths.append(sim)
    cond = cond.cpu().detach().numpy()
    cond = cond.reshape(cond.shape[1], cond.shape[3])
    real_conds.append(cond)

real_paths = np.array(real_paths)
real_conds = np.array(real_conds)
print(real_paths.shape)
print(real_conds.shape)

real_vals = np.transpose(real_paths, (0,2,1))
print(real_vals.shape)
to_pkl(f'results/real_vals.pkl', real_vals)

[*********************100%%**********************]  1 of 1 completed


X_train's shape is (4328, 1, 1, 2), X_test's shape is (481, 1, 1, 2)
y_train's label shape is (4328, 1, 1, 43), y_test's label shape is (481, 1, 1, 43)
(481, 1, 43)
(481, 1, 2)
(481, 43, 1)


### Load synthetic data

In [74]:
from LoadSynthetic import *

In [75]:
def get_syn_vals(model_path=f'./logs/{LOGIR_NAME}/Model/checkpoint', n=10, **kwargs):
    syn_data = Synthetic_Dataset(model_path=model_path, n=n, dataset=real_data, seq_len=real_data.output_size, conditions_dim=real_data.X_train.shape[-1], **kwargs)
    dataloader = data.DataLoader(syn_data, batch_size=1, num_workers=1, shuffle=True, **kwargs)

    paths = []
    conds = []

    for i, (cond, sim) in enumerate(dataloader):
        sim = sim.cpu().detach().numpy()
        sim = sim.reshape(sim.shape[1], sim.shape[3])
        paths.append(sim)
        cond = cond.cpu().detach().numpy()
        cond = cond.reshape(cond.shape[1], cond.shape[3])
        conds.append(cond)
        
    paths = np.array(paths)
    conds = np.array(conds)
    vals = np.transpose(paths, (0,2,1))
    np.random.shuffle(vals)
    return vals

def get_syn_val_sets(nsets, **kwargs):
    val_sets = []
    for i in range(nsets):
        val_sets.append(get_syn_vals(**kwargs))
    return val_sets

In [76]:
syn_val_sets = get_syn_val_sets(nsets=NSETS)
to_pkl(f'results/{LOGIR_NAME}/syn_val_sets.pkl', syn_val_sets)

### Load benchmark data

In [52]:
from utils.utils import GBM_Simulator

In [53]:
def get_gbm_vals(n=10, **kwargs):
    simulator = GBM_Simulator(dataset=real_data, nsamples=n)
    paths = simulator.run()
    paths = np.reshape(paths, (-1,1,paths.shape[-1]))

    vals = np.transpose(paths, (0, 2, 1))
    np.random.shuffle(vals)
    return vals

def get_gbm_val_sets(nsets, **kwargs):
    val_sets = []
    for i in range(nsets):
        val_sets.append(get_syn_vals(**kwargs))
    return val_sets

In [54]:
gbm_val_sets = get_gbm_val_sets(nsets=NSETS)
to_pkl(f'results/gbm_val_sets.pkl', gbm_val_sets)

# Run evaluations

In [77]:
real_vals = from_pkl(f'results/real_vals.pkl')
syn_val_sets = from_pkl(f'results/{LOGIR_NAME}/syn_val_sets.pkl')
gbm_val_sets = from_pkl(f'results/gbm_val_sets.pkl')

In [78]:
eval_scores = {
    'js_pca': {
        'syn': [],
        'gbm': []
    },
    'js_tsne': {
        'syn': [],
        'gbm': []
    },
    'fid': {
        'syn': [],
        'gbm': []
    }
}

In [79]:
def js_div(real_vals, syn_vals, gbm_vals, mode, n_components=10, **kwargs):
    real, syn, gbm = dim_reduction([real_vals, syn_vals, gbm_vals], n_components=n_components, mode=mode, **kwargs)
    syn_score = js_divergence(real, syn, verbose=False)
    gbm_score = js_divergence(real, gbm, verbose=False)
    return syn_score, gbm_score

In [80]:
for syn_vals, gbm_vals in tqdm(zip(syn_val_sets, gbm_val_sets)):
    for iter in range(3):
        # JS-PCA
        syn_score, gbm_score = js_div(real_vals, syn_vals, gbm_vals, mode='pca')
        eval_scores['js_pca']['syn'].append(syn_score)
        eval_scores['js_pca']['gbm'].append(gbm_score)
        
        # JS-TSNE
        syn_score, gbm_score = js_div(real_vals, syn_vals, gbm_vals, mode='tsne')
        eval_scores['js_tsne']['syn'].append(syn_score)
        eval_scores['js_tsne']['gbm'].append(gbm_score)
    
    # FID
    syn_score = fid(real_vals, syn_vals)
    gbm_score = fid(real_vals, gbm_vals)
    eval_scores['fid']['syn'].append(syn_score)
    eval_scores['fid']['gbm'].append(gbm_score)

to_pkl(f'results/{LOGIR_NAME}/eval_scores.pkl', eval_scores)

0it [00:00, ?it/s]

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.002s...
[t-SNE] Computed neighbors for 1443 samples in 0.146s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.043239
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.151970
[t-SNE] KL divergence after 300 iterations: 1.257519
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 0.126s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.043239
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.151867
[t-SNE] KL divergence after 300 iterations: 1.176743
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 

1it [00:07,  7.46s/it]

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.001s...
[t-SNE] Computed neighbors for 1443 samples in 0.131s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.044197
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.915894
[t-SNE] KL divergence after 300 iterations: 1.142146
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 0.116s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.044197
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.916016
[t-SNE] KL divergence after 300 iterations: 1.142514
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 

2it [00:15,  7.96s/it]

...

18it [02:32,  8.51s/it]

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 0.118s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.043973
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.763893
[t-SNE] KL divergence after 300 iterations: 1.152760
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 0.124s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.043973
[t-SNE] KL divergence after 250 iterations with early exaggeration: 60.763439
[t-SNE] KL divergence after 300 iterations: 1.151872
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 

19it [02:41,  8.65s/it]

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 0.123s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.043885
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.026493
[t-SNE] KL divergence after 300 iterations: 1.142973
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 0.125s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1443
[t-SNE] Computed conditional probabilities for sample 1443 / 1443
[t-SNE] Mean sigma: 0.043885
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.026684
[t-SNE] KL divergence after 300 iterations: 1.142979
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1443 samples in 0.000s...
[t-SNE] Computed neighbors for 1443 samples in 

20it [02:50,  8.52s/it]


# Run hypothesis tests

In [81]:
from scipy.stats import ttest_ind
eval_scores = from_pkl(f'results/{LOGIR_NAME}/eval_scores.pkl')

In [82]:
def hypothesis_test(syn_scores, gbm_scores):
    syn_scores = np.array(syn_scores)
    gbm_scores = np.array(gbm_scores)
    print("Synthetic:")
    print(f"\tmean  = {syn_scores.mean()}")
    print(f"\tstdev = {syn_scores.std()}")
    print("Benchmark:")
    print(f"\tmean  = {gbm_scores.mean()}")
    print(f"\tstdev = {gbm_scores.std()}")

### PCA JS-Divergence

In [83]:
hypothesis_test(eval_scores['js_pca']['syn'], eval_scores['js_pca']['gbm'])

Synthetic:
	mean  = 1.6576441169490852
	stdev = 0.2065909741205673
Benchmark:
	mean  = 2.919550678308244
	stdev = 0.3297409229289862


### TSNE JS-Divergence

In [84]:
hypothesis_test(eval_scores['js_tsne']['syn'], eval_scores['js_tsne']['gbm'])

Synthetic:
	mean  = 0.11182642067340585
	stdev = 0.013656558060486339
Benchmark:
	mean  = 0.23721891164765802
	stdev = 0.03350305663559553


### FID Score

In [85]:
hypothesis_test(eval_scores['fid']['syn'], eval_scores['fid']['gbm'])

Synthetic:
	mean  = 0.004907154425217041
	stdev = 0.00026894137654578075
Benchmark:
	mean  = 0.009741208312549928
	stdev = 0.00032674720969441126
